# 정적 웹 페이지 크롤링 실습

## 1. 크롤링 허용 여부 확인

- 크롤링할 사이트 주소/robots.txt 입력하여 정책 확인

![robots crawling 정책](../resource/W3/robotstxt.org.png)

### www.hollys.co.kr/robots.txt 검토결과

User-agent: *

Disallow: /membership

Disallow: /myHollys

## 2. 웹 페이지 분석하기

(1) 대상 웹: www.hollys.co.kr

        우측 하단의 전국 Store 검색

(2) HTML 코드 확인하기

        Ctrl + U: HTML로 열어보기
        <tbody> ~ </tbody>: 매장 정보 테이블
        <td>[0] 매장이 있는 지역
        <td>[1] 매장명
        <td>[3] 매장주소
        <td>[5] 전화번호

(3) 나머지 매장정보 확인하기

        한 페이지는 10개 매장 정보
        페이지 이동 -> 주소 url에 pageNo=2, 3, 4, ...
        마지막 페이지 58

## 3. 파이썬 셀 창에서 크롤링하기

- 크롤링 작업을 단계적으로 수행하며 확인

In [1]:
from bs4 import BeautifulSoup
import urllib.request

In [2]:
result = []

(1) 1 ~ 58 페이지까지 반복해서 url 설정

(2) url 요청하여 응답받은 웹 페이지 저장

(3) BeautifulSoup 객체 생성

(4) tr 태그 하위 td 태그 중에서 필요한 항목 추출하여 result 리스트에 저장

tbody > tr > td

In [3]:
for page in range(1, 59):
    Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=%d&sido=&gugun=&store=' % page
    print(Hollys_url)

    html = urllib.request.urlopen(Hollys_url)
    soupHollys = BeautifulSoup(html, 'html.parser')
    tag_tbody = soupHollys.find('tbody')
    for store in tag_tbody.find_all('tr'):
        if len(store) <= 3:  # 마지막 tr인 경우 매장 정보가 없음 -> 크롤링 중단
            print(store)
            break

        store_td = store.find_all('td')
        store_name = store_td[1].string
        store_sido = store_td[0].string
        store_address = store_td[3].string
        store_phone = store_td[5].string
        result.append([store_name]+[store_sido]+[store_address]+[store_phone])

https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=1&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=2&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=3&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=4&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=5&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=6&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=7&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=8&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=9&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=10&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=11&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=12&sido=&gugun=&store=
https://www.hollys.co.kr/store/korea/korStore2.do

(5) 크롤링된 결과의 확인

In [4]:
len(result)

482

In [6]:
result[0]

['경상국립대학생회관점', '경남 진주시', '경상남도 진주시 진주대로 501 경상국립대 학생회관 1층', '055-772-0931']

In [9]:
result[481]

['신촌점', '서울 서대문구', '서울특별시 서대문구 연세로 34 (창천동 31-12)  할리스', '02-393-2004']

In [10]:
store_td

[<td class="noline center_t">서울 서대문구</td>,
 <td class="center_t"><a href="#" onclick="javascript:storeView(1); return false;">신촌점</a></td>,
 <td class="center_t tdp0">영업중</td>,
 <td class="center_t"><a href="#" onclick="javascript:storeView(1); return false;">서울특별시 서대문구 연세로 34 (창천동 31-12)  할리스</a></td>,
 <td class="center_t">
 </td>,
 <td class="center_t">02-393-2004</td>]

In [11]:
store_td[1].string

'신촌점'

In [12]:
store_td[0].string

'서울 서대문구'

In [13]:
store_td[3].string

'서울특별시 서대문구 연세로 34 (창천동 31-12)  할리스'

In [14]:
store_td[5].string

'02-393-2004'

(6) 크롤링한 데이터 저장하기

- csv 파일 형태로 저장
- row, column의 테이블 형식으로 저장

In [15]:
import pandas as pd

hollys_tbl = pd.DataFrame(result, columns = ('store', 'sido-gu', 'address', 'phone'))

In [16]:
hollys_tbl.head()

,store,sido-gu,address,phone
0,경상국립대학생회관점,경남 진주시,경상남도 진주시 진주대로 501 경상국립대 학생회관 1층,055-772-0931
1,서서울공원점2,서울 양천구,서울특별시 양천구 남부순환로58길 37 신월동 205-36,070-4277-6756
2,전북대 후생관점,전북 전주시 덕진구,전북특별자치도 전주시 덕진구 백제대로 567 후생관 1층,063-270-2174
3,전북대 진수당점,전북 전주시 덕진구,전북 전주시 덕진구 백제대로 567 (전북대 진수당 연구동 1층) .,063-270-4350
4,전북대 중도라운지점,전북 전주시 덕진구,전북 전주시 덕진구 백제대로 567 중앙도서관 1층,.


In [17]:
hollys_tbl.tail()

,store,sido-gu,address,phone
477,합정역점,서울 마포구,서울특별시 마포구 양화로 36 (합정동 374-1) 할리스,02-6204-1234
478,부산달맞이점,부산 해운대구,"부산광역시 해운대구 달맞이길 199, 2~3층 (중동 1488-14) 할리스",051-731-3410
479,신림점,서울 관악구,서울특별시 관악구 신림로 353-1,02-877-0019
480,태평로점,서울 중구,"서울특별시 중구 세종대로 64, 해남빌딩 1층 (태평로2가 70-5) 할리스.",02-755-7795
481,신촌점,서울 서대문구,서울특별시 서대문구 연세로 34 (창천동 31-12) 할리스,02-393-2004


In [18]:
hollys_tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   store    482 non-null    object
 1   sido-gu  482 non-null    object
 2   address  482 non-null    object
 3   phone    481 non-null    object
dtypes: object(4)
memory usage: 15.2+ KB


In [19]:
hollys_tbl.describe()

,store,sido-gu,address,phone
count,482,482,482,481
unique,482,161,482,455
top,경상국립대학생회관점,서울 강남구,경상남도 진주시 진주대로 501 경상국립대 학생회관 1층,.
freq,1,15,1,21


In [24]:
hollys_tbl.to_csv("../resource/W3/output/hollys_utf8.csv", encoding='utf8', mode='w', index = True)
hollys_tbl.to_csv("../resource/W3/output/hollys_cp949.csv", encoding='cp949', mode='w', index = True)